In [121]:
import numpy as np
import matplotlib.pyplot as plt
from numpy.typing import NDArray
from scipy.special import comb

In [122]:
# physical parameters: optical linewidth (gamma), spin coherence time of emitter (t_coh), ancilla coherence time
# Global variables in uppercase
GAMMA = 2e9*2*np.pi # Hz
T_SPIN_COHERENCE = 13e-3 # s
T_ANCILLA_COHERENCE = np.inf

# Table 1
# loss: loss probability 

class Miu:
    def __init__(self, L, L_att, L_delay, m, L_feedback=np.nan):
        self._L = L
        self._L_att = L_att
        self.L_delay = L_delay  # does this depends on the position of qubit?
        self.m = m
        self.L_f = L_feedback
        self.coup = 0.05
        self.int_ancilla = 0
        self.depth = DEPTH

    @property
    def ext_tree(self):
        return 1 - np.exp(-self._L/(self.m+1)/self._L_att)
    @property
    def ext_RGS(self):
        return 1 - np.exp(-self._L/2/(self.m+1)/self._L_att)
    
    def int_feedback(self, no_feedback): # not sure how to implement this
        return 1 - np.exp(-self.L_f*no_feedback/self._L_att)
    
    @property
    def delay(self):
        return  1 - np.exp(-self.L_delay/self._L_att)

    def total_tree_a(self, apply):
        if apply:
            return 1 - (1-self.ext_tree)*(1-self.coup)*(1-self.int_ancilla)*(1-self.delay)
        else:
            return 1 - (1-self.ext_tree)*(1-self.coup)*(1-self.int_ancilla) 

    def total_tree_f(self, apply, no_feedback):
        if apply:
            return 1 - (1-self.ext_tree)*(1-self.coup)*(1-self.int_feedback(no_feedback))*(1-self.delay)
        else:
            return 1 - (1-self.ext_tree)*(1-self.coup)*(1-self.int_feedback(no_feedback)) 


EPSILON_DEPOLARIZATION = 5e-5
BETA = 500.

# time for operations:
class Time:
    def __init__(self, gamma=GAMMA, beta=BETA):
        self._gamma = gamma
        self._beta = beta 

        self.H = 100e-12  # s
        self.CZ_a = 100e-9 # s
        self.CZ_f = self.E_f # s
    
    @property
    def gamma(self):
        return self._gamma
    @property
    def P_a(self):
        return 1/self._gamma
    @property
    def P_f(self):
        return 1/self._gamma
    @property
    def M(self):
        return 10*self.P_a
    @property
    def E_a(self):    
        return self.P_a + self.H + self.M
    @property
    def E_f(self):
        return self._beta*self.P_f + self.H + self.M

t = Time(gamma=GAMMA)    
# setup parameters: distance between A&B (L), attenuation distance (L_att), speed of light in delay line (v_delay)
L = 100e3 # m
L_ATT = 20e3 # m
V_DELAY = 3e8 # m/s

# variable paramaters: m, branching parameter {b_i}
M = 500
DEPTH = 1
BRANCH_PARAM = np.zeros(DEPTH)

In [123]:
# tree properties
class Tree:
    def __init__(self, branch_param: NDArray, miu: Miu) -> None:
        self.b = branch_param # array 
        self.depth = len(self.b)
        self.miu = miu # same for all photons? Assume No

        self.n_total = self.num_qubits(1, self.depth)

        # Probability of obtaining an outcome from an indirect Z measurement of any photon in i-th level of the tree (eq. 5)
        # Here is 1 indexed (same as paper)
        self.R = np.zeros(self.depth)
        self.R[0] = np.nan

        # 
        # miu = self.miu.total_tree_a(apply=True)
        # miu1 = self.miu.total_tree_a(apply=True)

        miu = self.miu_total(delay_apply=True)
        miu1 = miu
        
        for i in range(self.depth-1, 0, -1): # d-1 ... 1
            if i == self.depth-1: 
                R_i_plus_2 = 0
                b_i_plus_1 = 0 # no qubits at depth+1 level
            elif i == self.depth-2: 
                R_i_plus_2 = 0 # qubits at the last level cannot be indirectly measured
                b_i_plus_1 = self.b[i+1] 
            else:
                b_i_plus_1 = self.b[i+1] 
                R_i_plus_2 = self.R[i+2] 
            
            # print(R_i_plus_2)
            # print(b_i_plus_1)
            self.R[i] = 1 - (1 - (1-miu)*(1-miu+miu*R_i_plus_2)**b_i_plus_1)**self.b[i] # confirm this

        # Success probability between neighbouring nodes (eq. 3, without ^m+1)
        if self.depth >= 3:
            R_2 = self.R[2] 
        else:   
            R_2 = 0
        self.P_succ = ((1-miu1+miu1*self.R[1])**self.b[0] - (miu1*self.R[1])**self.b[0])*(1-miu+miu*R_2)**self.b[1]
    
    

    # Number of qubits from depth i to j 
    def num_qubits(self, start: int, end: int) -> int:
        return np.sum([np.product(self.b[0:i]) for i in range(start, end+1)])
    

    def miu_list(self, apply_list) -> list:
        pass

    def miu(self, apply) -> float:
        pass
        # return self.miu.total_tree_a(apply)

    # Generation time (s)
    def T_tree(self, time: Time) -> float:
        pass
       
    
 
 
 # Generation time (s)
def T_tree_a(tree: Tree, time: Time) -> float:
    num_last_layer = tree.num_qubits(tree.depth,tree.depth)
    num_2nd_to_d_minus_1 = tree.num_qubits(2,tree.depth-1)
    num_1st_to_d_minus_1 = tree.num_qubits(1,tree.depth-1)
    return num_last_layer*time.P_a + (time._beta*tree.b[0] + num_2nd_to_d_minus_1)*time.E_a + num_1st_to_d_minus_1*time.CZ_a



In [124]:
# Ancilla and Feedback Schemes changes
class Tree_ancilla(Tree):
    def __init__(self, branch_param: NDArray, miu: Miu) -> None:
        super().__init__(branch_param, miu)
    
    def miu_total(self, delay_apply) -> float:
        return self.miu.total_tree_a(delay_apply)

    def T_tree(self, time:Time) -> float:
        num_last_layer = self.num_qubits(self.depth,self.depth)
        num_2nd_to_d_minus_1 = self.num_qubits(2,self.depth-1)
        num_1st_to_d_minus_1 = self.num_qubits(1,self.depth-1)
        return num_last_layer*time.P_a + (time._beta*self.b[0] + num_2nd_to_d_minus_1)*time.E_a + num_1st_to_d_minus_1*time.CZ_a
    
class Tree_feedback(Tree):
    def __init__(self, branch_param: NDArray, miu: Miu) -> None:
        super().__init__(branch_param, miu)
    
    def miu_total(self, delay_apply) -> float:
        no_feedback = 0 # Assume the worst case for all qubits
        return self.miu.total_tree_f(delay_apply, no_feedback)
    
    def T_tree(self, time: Time) -> float:
        num_last_layer = self.num_qubits(self.depth,self.depth)
        num_1st_to_d_minus_1 = self.num_qubits(1,self.depth-1)
        return num_last_layer*time.P_f + (time._beta*self.b[0] + num_1st_to_d_minus_1)*time.E_f + num_1st_to_d_minus_1*time.CZ_f 

In [125]:
# Error from single-photon measurement
class Error:
    def __init__(self, tree:Tree, time:Time,  t_spin_coherence, epsilon_depolarization=EPSILON_DEPOLARIZATION):
        self.epsilon_decoh =  3/4*(1-np.exp(-tree.T_tree(time)/t_spin_coherence/tree.n_total))
        self.epsilon_sp_measure = 2/3*(self.epsilon_decoh + epsilon_depolarization - 4/3*self.epsilon_decoh*epsilon_depolarization) # error of single photon measurement
        

        miu = tree.miu_total(delay_apply=True)
        miu1 = tree.miu_total(delay_apply=True)
        print("miu:", miu)

        self.tree = tree
        self.R = np.zeros(tree.depth)
        self.R[0] = np.nan


        # e_I_k: average error probablity of indirect measurements on a qubit A in the kth level
        self.e_I_k = np.zeros(tree.depth)
        self.e_I_k[0] = np.nan

        for k in range(tree.depth-1, 0, -1):


            if k == tree.depth-1: # no qubits at depth+1 level
                R_k_plus_2 = 0.
                b_k_plus_1 = 0
                e_I_k_plus_2 = 0.
            elif k == tree.depth-2: # qubits at the last level cannot be indirectly measured
                b_k_plus_1 = tree.b[k+1] 
                R_k_plus_2 = 0.
                e_I_k_plus_2 = 0.
            else:
                b_k_plus_1 = tree.b[k+1] 
                R_k_plus_2 = tree.R[k+2] 
                e_I_k_plus_2 = self.e_I_k[k+1] 

            
            # e_I_k|B: error prob from measurements of any qubit B and its children
            e_I_k_B = 0.
            for lk in range(b_k_plus_1+1): # lk: # of qubits in k+2 level that can only be directly measured
                counting = comb(b_k_plus_1, lk)
                # single_qubit_successful_indirect_measurement_given_can_be_measured 
                p  = R_k_plus_2/(1-miu+miu*R_k_plus_2)
                parity_error_prob = 0.5 * (1-(1-2*self.epsilon_sp_measure)**(1+lk)*(1-2*e_I_k_plus_2)**(b_k_plus_1-lk)) # how
                e_I_k_B += counting * (1-p)**lk * p**(b_k_plus_1-lk) * parity_error_prob
            
            # majority vote over measuring outcomes of mk qubits B1 ... B_mk
            S_k = (1-miu)*(1-miu+miu*R_k_plus_2)**b_k_plus_1 # A7

            def T_k(mk: int): #A9
                return comb(tree.b[k], mk) * S_k**mk * (1-S_k)**(tree.b[k]-mk)
            
            def e_I_k_mk(mk: int): # A10
                if mk%2 == 1:
                    return np.sum([comb(mk, j) * (e_I_k_B)**j * (1-e_I_k_B)**(mk-j) for j in range(int(np.ceil(mk/2)), mk+1)])
                else:
                    return np.sum([comb(mk-1, j) * (e_I_k_B)**j * (1-e_I_k_B)**(mk-1-j) for j in range(int(np.ceil(mk/2)), mk)])
                    # return np.sum([comb(mk, j) * (e_I_k_B)**j * (1-e_I_k_B)**(mk-j) for j in range(int(np.ceil(mk/2)), mk+1)])
            
            self.R[k] = np.sum([T_k(mk) for mk in range(1, tree.b[k]+1)]) # is this equal to tree.R[k]? #A12
            
            self.e_I_k[k] = np.sum([T_k(mk)*e_I_k_mk(mk) for mk in range(1, tree.b[k]+1)])/self.R[k] #A11




    # e_incorrect: average probability of incorrect decoding
        b0 = tree.b[0]
        b1 = tree.b[1]
        R1 = tree.R[1]
        
        if tree.depth >= 3:
            R2 = tree.R[2]
        else:
            R2 = 0.

        e_incorrect = 0.
        for l in range(b0 - 1 + 1):
            for n in range(b0 - l + 1):
                for m in range(b1 + 1):
                    counting = comb(b0, l)*comb(b0-l, n)*comb(b1, m)
                    successfully_decode_prob = (miu1*R1)**l * ((1-miu1)*(1-R1))**n * ((1-miu1)*R1)**(b0-l-n) * ((1-miu)*(1-R2))**m * R2**(b1-m)
                    e_incorrect += counting * successfully_decode_prob * self.e_n_m(n,m)
        self.e_incorrect = e_incorrect

    # n: 1st level qubits that can only be directly measured
    # m: 2nd level qubits that can only be directly measured 
    def e_n_m(self, n, m):
        
        e_I_1 = self.e_I_k[1]
        if self.tree.depth > 2:
            e_I_2 = self.e_I_k[2]
        else:
            e_I_2 = 0
        e_m = self.epsilon_sp_measure
        # correct parity for 1st level qubits
        b0 = self.tree.b[0]
        b1 = self.tree.b[1]
        correct_parity_prob_n = 1
        for i in range(n+1): # i: # of incorrectly measured (direct) qubits in 1st level 
            direct = comb(n, i) * e_m **i * (1-e_m)**(n-i)
            # j: # of incorrectly measured (indirect) qubits in 1st level 
            # i + j must be odd so that the overall parity is wrong
            j_list = np.arange(b0-1-n + 1)
            j_list = j_list[np.where((i+j_list) % 2 ==1)] # Why must 0 be included?
            indirect = 0
            for j in j_list:
                indirect += comb(b0 - 1 - n, j)* e_I_1**j * (1-e_I_1)**(b0-1-n-j)
            correct_parity_prob_n -= direct*indirect
        
       # correct parity for 2nd level qubits 
        correct_parity_prob_m = 1
        for i in range(m+1): # i: # of incorrectly measured (direct) qubits in 1st level 
            direct = comb(m, i) * e_m **i * (1-e_m)**(m-i)
            # j: # of incorrectly measured (indirect) qubits in 1st level 
            # i + j must be odd so that the overall parity is wrong
            j_list = np.arange(b1-m + 1)
            j_list = j_list[np.where((i+j_list) % 2 ==1)] # Why must 0 be included?
            indirect = 0
            for j in j_list:
                indirect += comb(b1 - 1 - m, j)* e_I_2**j * (1-e_I_2)**(b1-m-j)
            correct_parity_prob_m -= direct*indirect
    
        return 1 - (1-e_m)*correct_parity_prob_n*correct_parity_prob_m

In [126]:
# effective secret key rate (R_eff) (eq. 1)
# Binary entropy function
def h(x): 
    return  - x*np.log2(x) - (1-x)*np.log2(1-x)
# secret fraction for six-state protocol
def key_rate(F): # invalid when F is too small
    return F - h(1-F) - F*h((3*F-1)/(2*F))

def effective_key_rate(tree: Tree, time: Time, error: Error, M: int, n: int):
    fidelity = (1-error.e_incorrect/tree.P_succ)**(M+1)
    print("fidelity:", fidelity)
    r = key_rate(fidelity)  
    return r*tree.P_succ**(M+1)/T_tree_a(tree, time)/M/n*L/L_ATT


In [127]:
GAMMA = np.array([2e9, 100e9, 170e6, 100e9]) * 2 * np.pi
T_SPIN_COHERENCE = [13e-3, 4e-6, 1, 1]
L_neighbour = [1.7e3, 30.3e3, 1.8e3, 1.7e3]
BRANCH_PARAM = [np.array([4,16,5]), np.array([1,1,19]), np.array([4,18,5]), np.array([4,16,5])]
L_delay = [398, 80.4, 627.9, 380.7]

L = 1000e3
L_ATT = 20e3

for i in range(len(GAMMA)):
    M = L/L_neighbour[i] - 1
    miu = Miu(L, L_ATT, L_delay=L_delay[i], m=M)
    tree = Tree_ancilla(BRANCH_PARAM[i], miu=miu)
    time = Time(GAMMA[i], 500)
    error = Error(tree, time, t_spin_coherence=T_SPIN_COHERENCE[i])
    print("R_eff: ", effective_key_rate(tree, time, error, M, 2))


miu: 0.14460616843671803
fidelity: 0.9655095359707675
R_eff:  1893.4979026234205
miu: 0.7920199868218668
fidelity: 0.9494048777766362
R_eff:  3.375379829905646e-17
miu: 0.15860015697311258
fidelity: 0.9652121795080344
R_eff:  499.9846893474021
miu: 0.14386593266660042
fidelity: 0.966736977301539
R_eff:  2452.583382700242


In [134]:
GAMMA = np.array([2e9, 100e9, 170e6, 100e9]) * 2 * np.pi
T_SPIN_COHERENCE = [13e-3, 4e-6, 1., 1.]
L_neighbour = [1.1e3, 2.2e3, 27e3, 1.9e3]
BRANCH_PARAM = [np.array([4,16,5]), np.array([4,22,6]), np.array([1,1,18]), np.array([4,15,5])]
L_feedback = [540.3, 16.5, 93.6, 11.3]
L_delay = [1.2e3, 37.1, 378., 25.6]

L = 1000e3
L_ATT = 20e3

for i in range(4):
    M = L/L_neighbour[i] - 1
    miu = Miu(L, L_ATT, L_delay=L_delay[i], m=M, L_feedback=L_feedback[i])
    tree = Tree_ancilla(BRANCH_PARAM[i], miu=miu)
    time = Time(GAMMA[i], 500)
    error = Error(tree, time, t_spin_coherence=T_SPIN_COHERENCE[i])

    print("R_eff: ", effective_key_rate(tree, time, error, M, 1))




miu: 0.15320216328851022
fidelity: 0.9395802261526991
R_eff:  917.6334114292987
miu: 0.15053479184905028
fidelity: 0.26184487296799297
R_eff:  nan
miu: 0.7583327005505665
fidelity: 0.9987611552661344
R_eff:  2.9507692099355824e-18
miu: 0.1372008023349609
fidelity: 0.9709647449467383
R_eff:  6874.26664159275


/var/folders/5m/d9qdltz940j7ch1nc4sgtqf00000gn/T/ipykernel_22594/1537203938.py:4: RuntimeWarning: invalid value encountered in log2
  return  - x*np.log2(x) - (1-x)*np.log2(1-x)
